In [ ]:
# gurobi
import gurobipy as gp
from gurobipy import GRB

from itertools import combinations
import numpy as np
import pandas as pd

In [ ]:
Beta =  pd.read_csv('coef_test.csv') # read the updated coefficients
Beta = Beta.iloc[:, 1:] # remove the intercept 

#  set parameters phase
N = len(Beta)  # the number of features # 
n = int(N + N*(N-1)/2)    # the number of variables

# Create a new model
m = gp.Model("quadratic")

# Create variables
x = m.addVars(N , vtype=GRB.BINARY, name="x") # 

# Solution
sol = pd.DataFrame()
obj = pd.DataFrmae() 

# LOOP 
for i in range(N): # N=16 
    # convert Beta to Q matrix
    coef = np.array(Beta.loc[i])
    Q = create_upper_matrix(coef, N)

    # Set objective
    obj = gp.quicksum(Q[i,j] * x[i] * x[j] for i in x.keys() for j in x.keys() if i <= j)
    m.setObjective(obj, GRB.MAXIMIZE)
    
    try:
        # Optimize model
        m.optimize()
        
        x_new = [] # 
        
        for v in m.getVars():
            #print('%s %g' % (v.VarName, v.X))  
            x_new.append(x_new, v.X)
                
        sol = sol.append(sol, x_new) 
        obj = obj.append(obj, m.ObjVal)
                
        #print('Obj: %g' % m.ObjVal)

    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')
               

In [ ]:
sol # optimal combination of x
obj # Objective value